In [25]:
import logging
from clickhouse_driver import Client
from dotenv import load_dotenv
import os

# Загрузка переменных окружения
load_dotenv()

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("update_balances.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Параметры подключения
clickhouse_host = os.getenv('CLICKHOUSE_HOST', '10.95.19.132')
clickhouse_user = os.getenv('CLICKHOUSE_USER', 'default')
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = os.getenv('CLICKHOUSE_DB', 'default')

# Создание клиента ClickHouse
client = Client(
    host=clickhouse_host,
    user=clickhouse_user,
    password=clickhouse_password,
    port=9000,
    secure=False,
    settings={'strings_encoding': 'utf-8'}
)


def calculate_and_update_balances():
    """
    Рассчитывает и обновляет balance_mi8t, balance_mi17, balance_total, 
    stock_mi8t, stock_mi17, stock_total (с учётом stock_empty)
    для первой даты в таблице OlapCube_VNV.
    """
    try:
        # Извлечь первую дату
        query_first_date = f"""
        SELECT MIN(Dates) AS first_date
        FROM {database_name}.OlapCube_VNV
        """
        first_date_result = client.execute(query_first_date)
        first_date = first_date_result[0][0]
        logger.info(f"Первая дата в OlapCube_VNV: {first_date}")

        # Проверка наличия первой даты
        if not first_date:
            logger.error("Не удалось получить первую дату. Завершаем выполнение.")
            return

        # Рассчитать balance_mi8t
        query_balance_mi8t = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE ac_typ = 'Ми-8Т' AND Status_P = 'Эксплуатация' AND Dates = '{first_date}'
        """
        balance_mi8t_count = client.execute(query_balance_mi8t)[0][0]

        # Получить mi8t_count
        query_mi8t_count = f"""
        SELECT mi8t_count 
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{first_date}'
        LIMIT 1
        """
        mi8t_count_result = client.execute(query_mi8t_count)
        if mi8t_count_result:
            mi8t_count = mi8t_count_result[0][0]
            balance_mi8t = balance_mi8t_count - mi8t_count
            logger.info(f"balance_mi8t: {balance_mi8t} (balance_mi8t_count: {balance_mi8t_count} - mi8t_count: {mi8t_count})")
        else:
            logger.warning(f"mi8t_count не найден для даты {first_date}. Устанавливаем balance_mi8t = balance_mi8t_count.")
            balance_mi8t = balance_mi8t_count

        # Рассчитать balance_mi17
        query_balance_mi17 = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE ac_typ = 'Ми-17' AND Status_P = 'Эксплуатация' AND Dates = '{first_date}'
        """
        balance_mi17_count = client.execute(query_balance_mi17)[0][0]

        # Получить mi17_count
        query_mi17_count = f"""
        SELECT mi17_count 
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{first_date}'
        LIMIT 1
        """
        mi17_count_result = client.execute(query_mi17_count)
        if mi17_count_result:
            mi17_count = mi17_count_result[0][0]
            balance_mi17 = balance_mi17_count - mi17_count
            logger.info(f"balance_mi17: {balance_mi17} (balance_mi17_count: {balance_mi17_count} - mi17_count: {mi17_count})")
        else:
            logger.warning(f"mi17_count не найден для даты {first_date}. Устанавливаем balance_mi17 = balance_mi17_count.")
            balance_mi17 = balance_mi17_count

        # Рассчитать balance_total
        balance_total = balance_mi8t + balance_mi17
        logger.info(f"balance_total: {balance_total} (balance_mi8t: {balance_mi8t} + balance_mi17: {balance_mi17})")

        # Рассчитать stock_mi8t
        query_stock_mi8t = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE ac_typ = 'Ми-8Т' AND Status_P = 'Исправен' AND Dates = '{first_date}'
        """
        stock_mi8t = client.execute(query_stock_mi8t)[0][0]
        logger.info(f"stock_mi8t: {stock_mi8t}")

        # Рассчитать stock_mi17
        query_stock_mi17 = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE ac_typ = 'Ми-17' AND Status_P = 'Исправен' AND Dates = '{first_date}'
        """
        stock_mi17 = client.execute(query_stock_mi17)[0][0]
        logger.info(f"stock_mi17: {stock_mi17}")

        # Рассчитать stock_empty (ac_typ пусто и Status_P = 'Исправен')
        query_stock_empty = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE (ac_typ = '' OR ac_typ IS NULL) AND Status_P = 'Исправен' AND Dates = '{first_date}'
        """
        stock_empty = client.execute(query_stock_empty)[0][0]
        logger.info(f"stock_empty: {stock_empty}")

        # Рассчитать stock_total с учётом stock_empty
        stock_total = stock_mi8t + stock_mi17 + stock_empty
        logger.info(f"stock_total: {stock_total} (stock_mi8t: {stock_mi8t} + stock_mi17: {stock_mi17} + stock_empty: {stock_empty})")

        # Обновить таблицу OlapCube_VNV
        update_query = f"""
        ALTER TABLE {database_name}.OlapCube_VNV 
        UPDATE 
            balance_mi8t = {balance_mi8t},
            balance_mi17 = {balance_mi17},
            balance_total = {balance_total},
            stock_mi8t = {stock_mi8t},
            stock_mi17 = {stock_mi17},
            stock_total = {stock_total}
        WHERE Dates = '{first_date}'
        """
        client.execute(update_query)
        logger.info("Баланс и запасы успешно обновлены для первой даты.")

    except Exception as e:
        logger.error(f"Ошибка при расчете или обновлении балансов: {e}", exc_info=True)


def main():
    calculate_and_update_balances()


if __name__ == "__main__":
    main()


2024-12-02 11:03:50,424 - INFO - Первая дата в OlapCube_VNV: 2024-11-25
2024-12-02 11:03:50,466 - INFO - balance_mi8t: -11.0 (balance_mi8t_count: 75 - mi8t_count: 86.0)
2024-12-02 11:03:50,507 - INFO - balance_mi17: 17.0 (balance_mi17_count: 87 - mi17_count: 70.0)
2024-12-02 11:03:50,508 - INFO - balance_total: 6.0 (balance_mi8t: -11.0 + balance_mi17: 17.0)
2024-12-02 11:03:50,533 - INFO - stock_mi8t: 3
2024-12-02 11:03:50,556 - INFO - stock_mi17: 5
2024-12-02 11:03:50,578 - INFO - stock_empty: 1
2024-12-02 11:03:50,579 - INFO - stock_total: 9 (stock_mi8t: 3 + stock_mi17: 5 + stock_empty: 1)
2024-12-02 11:03:50,587 - INFO - Баланс и запасы успешно обновлены для первой даты.
